# 02: Generate gridded WBGT in the shade estimates
*Use downscaled CMIP6 projections from the [NEX-GDDP-CMIP6 dataset](https://www.nccs.nasa.gov/services/data-collections/land-based-products/nex-gddp-cmip6) to generate gridded estimates of WBGT. The projections cover historical and future (SSP2-4.5) periods at a daily timestep and 0.25 degree resolution for the entire globe's land surface.*

In [ ]:
import os

import coiled
import dask
import numpy as np
import thermofeel as tf
import xarray as xr
import xclim
from utils import gcm_list, load_nasanex, wbgt

os.environ["USE_PYGEOS"] = "0"

In [ ]:
def adjust_pressure(temperature, elevation):
    """
    Approximate surface pressure given the elevation and temperature.
    Method from https://doi.org/10.1038/s41598-019-50047-w
    """
    return 101325 * np.power(10, -elevation / (18400 * temperature / 273.15))

Set up cluster to handle multiprocessing using a Dask client.

In [ ]:
# cluster = coiled.Cluster(n_workers=3, worker_memory="64 GiB")
cluster = coiled.Cluster(
n_workers=10,
worker_vm_types=["m8g.4xlarge"],
scheduler_vm_types=["m8g.4xlarge"],
region="us-west-2",
spot_policy="spot_with_fallback")


cluster.adapt(minimum=3, maximum=30)

client = cluster.get_client()

Read in elevation data, which was processed in `01_elevation.ipynb`.

In [ ]:
elev = xr.open_zarr(
    "s3://carbonplan-climate-impacts/extreme-heat/v1.0/inputs/elevation.zarr"
)
elev = elev.chunk({"lat": -1, "lon": -1}).compute()

Identify which scenarios and years to evaluate.

In [ ]:
scenario_years = {
    "historical": np.arange(1985, 2015),
    "ssp245": np.arange(2015, 2091),
    "ssp370": np.arange(2015, 2091),
}

Calculate future projections of WBGT.

In [ ]:
generate_wbgt_projections = True
variables = ["tasmax", "huss", "tas"]
if generate_wbgt_projections:
    for gcm in gcm_list:
        for scenario, years in scenario_years.items():
            id_string = f"{gcm}-{scenario}"
            print(id_string)
            for year in years:
                print(year)
                output = (
                    f"s3://carbonplan-scratch/extreme-heat/wbgt-shade-"
                    f"gridded/years/{gcm}/{id_string}-{year}.zarr"
                )
                ds = load_nasanex(
                    gcm=gcm,
                    scenario=scenario,
                    variables=variables,
                    years=[year],
                    chunk_dict={"time": 40},
                )

                # calculate elevation-adjusted pressure
                ds["ps"] = xr.apply_ufunc(
                    adjust_pressure, ds["tas"], elev, dask="allowed"
                ).rename({"elevation": "ps"})["ps"]
                ds["ps"].attrs["units"] = "Pa"
                ds["hurs"] = xclim.indices.relative_humidity(
                    tas=ds["tasmax"], huss=ds["huss"], ps=ds["ps"]
                )
                ds["tasmax"].attrs = {}

                # windspeed assumption of 0.5 m/s (approximating shaded/indoor
                # conditions)
                ds["sfcWind"] = (ds["tas"] - ds["tas"]) + 0.5
                ds["WBT"] = tf.thermofeel.calculate_wbt(
                    ds["tasmax"] - 273.15, ds["hurs"]
                )

                ds["BGT"] = tf.thermofeel.calculate_bgt(
                    ds["tasmax"], ds["tasmax"], ds["sfcWind"]
                )
                ds["WBGT"] = wbgt(ds["WBT"], ds["BGT"], ds["tasmax"] - 273.15)
                ds["WBGT"].attrs["units"] = "degC"
                ds = ds[["WBGT"]]
                ds = dask.optimize(ds)[0]
                t = ds.to_zarr(output, consolidated=True, mode="w", compute=False)
                t.compute()